In [1]:
from datetime import datetime
import pandas as pd
import pyodbc
import requests
import os

In [2]:
rldc_url = 'https://wbes.nrldc.in/xml/'


In [3]:
rawtable = pd.read_html(rldc_url)

df = pd.DataFrame(rawtable[0])

df["Timestamp"] = pd.to_datetime(df.Timestamp, format="%m/%d/%Y %I:%M:%S %p")

df.head(5)

,Filename,Timestamp
0,FullSchedule-(182)-06-11-2021.xml,2021-11-06 20:33:47
1,FullSchedule-(181)-06-11-2021.xml,2021-11-06 20:22:57
2,FullSchedule-(180)-06-11-2021.xml,2021-11-06 20:18:32
3,FullSchedule-(4)-07-11-2021.xml,2021-11-06 20:07:59
4,FullSchedule-(179)-06-11-2021.xml,2021-11-06 20:07:41


In [6]:
df.dtypes

Filename             object
Timestamp    datetime64[ns]
dtype: object

In [33]:
print(rawtable)
finaldata=rawtable

[                               Filename           Timestamp
0     FullSchedule-(126)-28-10-2021.xml 2021-10-28 12:37:55
1     FullSchedule-(125)-28-10-2021.xml 2021-10-28 12:33:57
2     FullSchedule-(124)-28-10-2021.xml 2021-10-28 12:23:05
3     FullSchedule-(123)-28-10-2021.xml 2021-10-28 12:18:44
4     FullSchedule-(122)-28-10-2021.xml 2021-10-28 12:07:59
...                                 ...                 ...
8982   FullSchedule-(86)-15-09-2021.xml 2021-09-15 08:34:17
8983   FullSchedule-(85)-15-09-2021.xml 2021-09-15 08:23:43
8984   FullSchedule-(83)-15-09-2021.xml 2021-09-15 08:09:00
8985   FullSchedule-(82)-15-09-2021.xml 2021-09-15 08:04:28
8986   FullSchedule-(81)-15-09-2021.xml 2021-09-15 07:53:43

[8987 rows x 2 columns]]


In [34]:
for index, row in finaldata:
    print(index)
    print(row)
    #print("Downloading Record "+str(row.Filename))
    # print(str(row.Filename))
    download_url = rldc_url + row
    print(download_url)

    response = requests.get(download_url)
    print(response)

    folderpath = r"F:\flutegrid\NRLDC_Python_Example\NRLDC"
    filename = os.path.join(folderpath, row)
    print(filename)
    #print(row[2:])

    #print(response.content)

    with open(filename, 'wb') as file:
        file.write(response.content)
        #print(file)
    print("Done")



Filename
Timestamp
https://wbes.nrldc.in/xml/Timestamp
<Response [404]>
F:\flutegrid\NRLDC_Python_Example\NRLDC\Timestamp
Done


In [19]:
# Part 1 - Get Max Date from DB
from datetime import datetime
import pandas as pd
import pyodbc
import requests
import os


# Static Declarations.
server = '10.10.69.97'
database = 'NTPCPowerBI'
username = 'sa'
password = 'Ntpc@123'

rldc_url = 'https://wbes.nrldc.in/xml/'

# DataBase Connc

cnxn = pyodbc.connect('DRIVER={SQL Server};SERVER='+server +
                      ';DATABASE='+database+';UID='+username+';PWD=' + password)

cursor = cnxn.cursor()

# Get Data

max_time = cursor.execute(
    "select max(FILETIME) from NTPCPowerBI.dbo.RLDC_XML_PortingStatus with (nolock)")
max_time = cursor.fetchall()
data = [list(rows) for rows in max_time]
maxdbtime = data[0]
maxdbtime = pd.to_datetime(maxdbtime)


# Part 2 - Get Data From Website


def dataparser(url, dbtime):

    rawtable = pd.read_html(url)

    df = pd.DataFrame(rawtable[0])

    df["Timestamp"] = pd.to_datetime(
        df.Timestamp, format="%m/%d/%Y %I:%M:%S %p")

    # print(f'Min Time : {df["Timestamp"].min()}')
    # print(f'Max Time : {df["Timestamp"].max()}')

    # df['created_at'] = pd.to_datetime(
    #     datetime.now(), format="%Y-%m-%d %H:%M:%S")

    df['rldc'] = 'NRLDC'

    df['porting_status'] = 'N'

    finadf = df[df["Timestamp"] > dbtime]

    return finadf


finaldata = dataparser(rldc_url, maxdbtime[0])


if len(finaldata) < 1:
    print("No New  Records")
else:
    print(f'Total New Records : {len(finaldata)}')

# Part 3 - Insert Data into DB

insertquery = "INSERT INTO NTPCPowerBI.dbo.RLDC_XML_PortingStatus (FileName,FileTime,RLDC,Porting_Status) VALUES (?,?,?,'N')"
for index, row in finaldata.iterrows():
    print("Inserting Record "+str(row.Filename))
    cursor.execute(insertquery, row.Filename, row.Timestamp, row.rldc)
    cnxn.commit()
    # cursor.close()


# Part 4 - Download XML FIles

for index, row in finaldata.iterrows():
    print("Downloading Record "+str(row.Filename))
    # print(str(row.Filename))
    download_url = rldc_url + row.Filename

    response = requests.get(download_url)

    folderpath = "xmlfiles"
    filename = os.path.join(folderpath, row.Filename)

    # print(response.content)

    with open(filename, 'wb') as file:
        file.write(response.content)
    print("Done")


# Part 5 - Re-Download XML FIles
# Get Data

# dbfilenames = cursor.execute(
#     " SELECT [FileName] as Filenames  FROM [NTPCPowerBI].[dbo].[RLDC_XML_PortingStatus]  with (nolock) where [Porting_Status] = 'N'")
# dbfilenames = dbfilenames.fetchall()

# dbfilenamesdf = pd.DataFrame(dbfilenames, columns=['Filename'])

# print(dbfilenamesdf)

# for name in dbfilenames:
#     dbfilename = name[0]
#     print("Downloading XML File ..   "+dbfilename)

#     download_url = rldc_url + dbfilename

#     response = requests.get(download_url)

#     folderpath = "xmlfiles"

#     filename = os.path.join(folderpath, dbfilename)

#     # print(download_url)
#     # print(filename)
#     # print(response.content)

#     with open(filename, 'wb') as file:
#         file.write(response.content)
#     print("Done")


# Part 6 - Download XML FIles


OperationalError: ('08001', '[08001] [Microsoft][ODBC SQL Server Driver][DBNETLIB]SQL Server does not exist or access denied. (17) (SQLDriverConnect); [08001] [Microsoft][ODBC SQL Server Driver][DBNETLIB]ConnectionOpen (Connect()). (53)')